## Machine Learning Classifaction
1. Support Vector Machine SVM

In [50]:
from sklearn import svm
import pandas as pd
import matplotlib.pyplot as plt

#Since the advatnage of SVM is that it is 
#very effective in high dimensional spaces,
#We will not use PCA on the training sets 
#to reduce the number of dimensions for SVM

#From all the PhotoObj parameters, for the first run we'll be using all 
#parameters that are not RA, DEC and Errors
photoParam = open("./machine-learning-sets/PhotoObj_params_base.txt","r")

paramList = photoParam.readlines()
photoParam.close()

svmParam = open("./machine-learning-sets/SVM_params.txt","w+")

count = 0
for n in paramList:
    #Removes Err from list & the first 8 Parameters
    #which are not helpful with SVM
    if 'Err' not in n and count>8:
        svmParam.write('p.'+n)
    count+=1
    
svmParam.close()
svmParam = open('./machine-learning-sets/SVM_params.txt','r')
    
svmList = svmParam.readlines()
svmParam.close()

#No New Line svmParam text file to input to the CrossID SQL search
svmParam_no_nl = open("./machine-learning-sets/SVM_params_no_nl.txt","w+")

for n in svmList:
    svmParam_no_nl.write(n.replace('\n',''))

svmParam_no_nl.readlines()
    
svmParam_no_nl.close()

In [39]:
#Now we go back to SDSS crossid with this paramList & the two 
#foreground and background data set and retrieve all the
#parameters

#First need to extract only name, RA, DEC from the two data sets
#for SDSS CrossID
base_foreground = pd.read_csv('./complete/dwarfs.csv')
base_foreground.head()


,objID,ra,dec,run,rerun,camcol,field,type,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z,Redshift
0,1237680311772250464,16.765005,32.389555,8111,301,5,174,GALAXY,17.46509,16.28591,15.81043,15.55470,15.38543,0.016998
1,1237680315521237319,16.792826,32.346872,8112,301,4,35,GALAXY,18.74607,17.03549,16.28892,15.90131,15.58982,0.014160
2,1237680311772250198,16.685466,32.418586,8111,301,5,174,GALAXY,18.68646,17.02711,16.25918,15.87417,15.60222,0.020514
3,1237680315521302864,16.966691,32.256108,8112,301,4,36,GALAXY,18.19989,16.57700,15.77055,15.38624,15.10587,0.018193
4,1237680311772381486,17.066149,32.499011,8111,301,5,176,GALAXY,18.66414,16.91150,16.11959,15.70180,15.43588,0.018373


In [40]:
cross_id_foreground = base_foreground[['objID','ra','dec']]
cross_id_foreground.head()

,objID,ra,dec
0,1237680311772250464,16.765005,32.389555
1,1237680315521237319,16.792826,32.346872
2,1237680311772250198,16.685466,32.418586
3,1237680315521302864,16.966691,32.256108
4,1237680311772381486,17.066149,32.499011


In [41]:
base_background = pd.read_csv('./complete/background.csv')
base_background.head()

,ObjectName,objID,ra,dec,run,rerun,camcol,field,type,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z,Redshift,RedshiftFlag
0,IVZw038NOTES15,1237680316058304809,17.322279,32.611307,8112,301,5,38,GALAXY,19.58102,17.45923,16.43703,16.01473,15.67513,0.066308,SPEC
1,WHLJ010942.8+321639,1237680315521499274,17.428386,32.277482,8112,301,4,39,GALAXY,19.28566,17.39932,16.30085,15.79945,15.38015,0.114700,SPEC
2,2MASXJ01094280+3216390,1237680315521499274,17.428386,32.277482,8112,301,4,39,GALAXY,19.28566,17.39932,16.30085,15.79945,15.38015,0.114346,SPEC
3,2MASXJ01094004+3213539,1237680315521499270,17.416991,32.231698,8112,301,4,39,GALAXY,19.38819,17.43331,16.35997,15.89763,15.51898,0.114950,SPEC
4,CGCG501-105,1237680316058304582,17.335133,32.722856,8112,301,5,38,GALAXY,17.69355,25.21871,16.13597,15.38638,14.74868,0.034811,NaN


In [42]:
cross_id_background = base_background[['objID','ra','dec']]
cross_id_background.head()

,objID,ra,dec
0,1237680316058304809,17.322279,32.611307
1,1237680315521499274,17.428386,32.277482
2,1237680315521499274,17.428386,32.277482
3,1237680315521499270,17.416991,32.231698
4,1237680316058304582,17.335133,32.722856


In [43]:
#Exporting the two files to use in CROSSID
cross_id_foreground.to_csv(r'./machine-learning-sets/foreground_crossID.csv', index=False)
cross_id_background.to_csv(r'./machine-learning-sets/background_crossID.csv', index=False)

SQL used to grab all the PhotoObj parameters that are relevant based on the SVM_params_no_nl.txt file. This SQL works but be patient, it takes a while to grab all the parameter for each element.

```SQL
SELECT 
p.type,p.clean,p.probPSF,p.insideMask,p.flags,p.rowc,p.colc,p.rowv,p.colv,p.rowc_u,p.rowc_g,p.rowc_r,p.rowc_i,p.rowc_z,p.colc_u,p.colc_g,p.colc_r,p.colc_i,p.colc_z,p.sky_u,p.sky_g,p.sky_r,p.sky_i,p.sky_z,p.skyIvar_u,p.skyIvar_g,p.skyIvar_r,p.skyIvar_i,p.skyIvar_z,p.psfMag_u,p.psfMag_g,p.psfMag_r,p.psfMag_i,p.psfMag_z,p.fiberMag_u,p.fiberMag_g,p.fiberMag_r,p.fiberMag_i,p.fiberMag_z,p.fiber2Mag_u,p.fiber2Mag_g,p.fiber2Mag_r,p.fiber2Mag_i,p.fiber2Mag_z,p.petroMag_u,p.petroMag_g,p.petroMag_r,p.petroMag_i,p.petroMag_z,p.psfFlux_u,p.psfFlux_g,p.psfFlux_r,p.psfFlux_i,p.psfFlux_z,p.psfFluxIvar_u,p.psfFluxIvar_g,p.psfFluxIvar_r,p.psfFluxIvar_i,p.psfFluxIvar_z,p.fiberFlux_u,p.fiberFlux_g,p.fiberFlux_r,p.fiberFlux_i,p.fiberFlux_z,p.fiberFluxIvar_u,p.fiberFluxIvar_g,p.fiberFluxIvar_r,p.fiberFluxIvar_i,p.fiberFluxIvar_z,p.fiber2Flux_u,p.fiber2Flux_g,p.fiber2Flux_r,p.fiber2Flux_i,p.fiber2Flux_z,p.fiber2FluxIvar_u,p.fiber2FluxIvar_g,p.fiber2FluxIvar_r,p.fiber2FluxIvar_i,p.fiber2FluxIvar_z,p.petroFlux_u,p.petroFlux_g,p.petroFlux_r,p.petroFlux_i,p.petroFlux_z,p.petroFluxIvar_u,p.petroFluxIvar_g,p.petroFluxIvar_r,p.petroFluxIvar_i,p.petroFluxIvar_z,p.petroRad_u,p.petroRad_g,p.petroRad_r,p.petroRad_i,p.petroRad_z,p.petroR50_u,p.petroR50_g,p.petroR50_r,p.petroR50_i,p.petroR50_z,p.petroR90_u,p.petroR90_g,p.petroR90_r,p.petroR90_i,p.petroR90_z,p.q_u,p.q_g,p.q_r,p.q_i,p.q_z,p.u_u,p.u_g,p.u_r,p.u_i,p.u_z,p.mE1_u,p.mE1_g,p.mE1_r,p.mE1_i,p.mE1_z,p.mE2_u,p.mE2_g,p.mE2_r,p.mE2_i,p.mE2_z,p.mRrCc_u,p.mRrCc_g,p.mRrCc_r,p.mRrCc_i,p.mRrCc_z,p.mCr4_u,p.mCr4_g,p.mCr4_r,p.mCr4_i,p.mCr4_z,p.mE1PSF_u,p.mE1PSF_g,p.mE1PSF_r,p.mE1PSF_i,p.mE1PSF_z,p.mE2PSF_u,p.mE2PSF_g,p.mE2PSF_r,p.mE2PSF_i,p.mE2PSF_z,p.mRrCcPSF_u,p.mRrCcPSF_g,p.mRrCcPSF_r,p.mRrCcPSF_i,p.mRrCcPSF_z,p.mCr4PSF_u,p.mCr4PSF_g,p.mCr4PSF_r,p.mCr4PSF_i,p.mCr4PSF_z,p.deVRad_u,p.deVRad_g,p.deVRad_r,p.deVRad_i,p.deVRad_z,p.deVAB_u,p.deVAB_g,p.deVAB_r,p.deVAB_i,p.deVAB_z,p.deVPhi_u,p.deVPhi_g,p.deVPhi_r,p.deVPhi_i,p.deVPhi_z,p.deVMag_u,p.deVMag_g,p.deVMag_r,p.deVMag_i,p.deVMag_z,p.deVFlux_u,p.deVFlux_g,p.deVFlux_r,p.deVFlux_i,p.deVFlux_z,p.deVFluxIvar_u,p.deVFluxIvar_g,p.deVFluxIvar_r,p.deVFluxIvar_i,p.deVFluxIvar_z,p.expRad_u,p.expRad_g,p.expRad_r,p.expRad_i,p.expRad_z,p.expAB_u,p.expAB_g,p.expAB_r,p.expAB_i,p.expAB_z,p.expPhi_u,p.expPhi_g,p.expPhi_r,p.expPhi_i,p.expPhi_z,p.expMag_u,p.expMag_g,p.expMag_r,p.expMag_i,p.expMag_z,p.modelMag_u,p.modelMag_g,p.modelMag_r,p.modelMag_i,p.modelMag_z,p.cModelMag_u,p.cModelMag_g,p.cModelMag_r,p.cModelMag_i,p.cModelMag_z,p.expFlux_u,p.expFlux_g,p.expFlux_r,p.expFlux_i,p.expFlux_z,p.expFluxIvar_u,p.expFluxIvar_g,p.expFluxIvar_r,p.expFluxIvar_i,p.expFluxIvar_z,p.modelFlux_u,p.modelFlux_g,p.modelFlux_r,p.modelFlux_i,p.modelFlux_z,p.modelFluxIvar_u,p.modelFluxIvar_g,p.modelFluxIvar_r,p.modelFluxIvar_i,p.modelFluxIvar_z,p.cModelFlux_u,p.cModelFlux_g,p.cModelFlux_r,p.cModelFlux_i,p.cModelFlux_z,p.cModelFluxIvar_u,p.cModelFluxIvar_g,p.cModelFluxIvar_r,p.cModelFluxIvar_i,p.cModelFluxIvar_z,p.aperFlux7_u,p.aperFlux7_g,p.aperFlux7_r,p.aperFlux7_i,p.aperFlux7_z,p.aperFlux7Ivar_u,p.aperFlux7Ivar_g,p.aperFlux7Ivar_r,p.aperFlux7Ivar_i,p.aperFlux7Ivar_z,p.lnLStar_u,p.lnLStar_g,p.lnLStar_r,p.lnLStar_i,p.lnLStar_z,p.lnLExp_u,p.lnLExp_g,p.lnLExp_r,p.lnLExp_i,p.lnLExp_z,p.lnLDeV_u,p.lnLDeV_g,p.lnLDeV_r,p.lnLDeV_i,p.lnLDeV_z,p.fracDeV_u,p.fracDeV_g,p.fracDeV_r,p.fracDeV_i,p.fracDeV_z,p.flags_u,p.flags_g,p.flags_r,p.flags_i,p.flags_z,p.type_u,p.type_g,p.type_r,p.type_i,p.type_z,p.probPSF_u,p.probPSF_g,p.probPSF_r,p.probPSF_i,p.probPSF_z,p.ra,p.dec,p.cx,p.cy,p.cz,p.b,p.l,p.offsetRa_u,p.offsetRa_g,p.offsetRa_r,p.offsetRa_i,p.offsetRa_z,p.offsetDec_u,p.offsetDec_g,p.offsetDec_r,p.offsetDec_i,p.offsetDec_z,p.extinction_u,p.extinction_g,p.extinction_r,p.extinction_i,p.extinction_z,p.psffwhm_u,p.psffwhm_g,p.psffwhm_r,p.psffwhm_i,p.psffwhm_z,p.mjd,p.airmass_u,p.airmass_g,p.airmass_r,p.airmass_i,p.airmass_z,p.phioffset_u,p.phioffset_g,p.phioffset_r,p.phioffset_i,p.phioffset_z,p.nProf_u,p.nProf_g,p.nProf_r,p.nProf_i,p.nProf_z,p.loadVersion,p.htmID,p.fieldID,p.parentID,p.specObjID,p.u,p.g,p.r,p.i,p.z,p.err_u,p.err_g,p.err_r,p.err_i,p.err_z,p.dered_u,p.dered_g,p.dered_r,p.dered_i,p.dered_z,p.cloudCam_u,p.cloudCam_g,p.cloudCam_r,p.cloudCam_i,p.cloudCam_z,p.resolveStatus,p.thingId,p.balkanId,p.nObserve,p.nDetect,p.nEdge,p.score,p.calibStatus_u,p.calibStatus_g,p.calibStatus_r,p.calibStatus_i,p.calibStatus_z,p.nMgyPerCount_u,p.nMgyPerCount_g,p.nMgyPerCount_r,p.nMgyPerCount_i,p.nMgyPerCount_z,p.TAI_u,p.TAI_g,p.TAI_r,p.TAI_i,p.TAI_z
FROM #upload u
      JOIN #x x ON x.up_id = u.up_id
      JOIN PhotoObj p ON p.objID = x.objID 
ORDER BY x.up_id
```
